In [9]:
from bs4 import BeautifulSoup
import requests

res = {'title': [], 'date': [], 'url': []}

for i in range(1, 47):
    html = requests.get(f"https://sw7up.cbnu.ac.kr/community/notice?page={i}&limit=10&sort=-createdAt").text
    bsObject = BeautifulSoup(html, "html.parser")

    for link in bsObject.find_all('span', 'mb-0'):
        print(str(link).split('>')[1].split('<')[0])
        res['title'].append(str(link).split('>')[1].split('<')[0])

2022년 SW중심대학사업단 Git-Github 기초/심화 특강 신청 안내(~11/24)
2022학년도 2학기 마일리지 장학금 제출 안내(필독★)
코딩이력관리시스템 매뉴얼 다운로드
2022년 2차 Solve-UpⓇ 프로그래밍 경진대회 참가 신청 안내(~11/23)
2022학년도 2학기 마일리지 장학생 모집 안내 (신청일: 11/14~11/29)
(홍보) KOSA, 미래 유망 산업 공동 직무설명회 (11/16(수),14시~,온라인)
2022학년도 2학기 산학활동참여 마일리지 장학금 신청
2022-1기 일경험수련과정 참여신청서 제출안내
2022년 SW중심대학사업단 글로벌캡스톤디자인(CEDC 2022) 참가 신청 안내(~11/18)
2022학년도 2학기 SW융합전공 장학생 선발 결과 안내
2022년 제 18회 하반기 TOPCIT 정기평가 안내
2022년 현장실습(인턴십) 수기 공모전 수상자 안내
2022년 SW융합전공 체험수기 공모전 수상자 안내
2022 SW인재페스티벌 참가 우수 작품 공모전 모집 공고(10/4 ~ 10/17)
(홍보) 2022 SW인재페스티벌 행사(2022.12.08~12.09)
2022년 오픈소스SW 라이선스 교육 프로그램 참가 신청 안내(~10/27)
(홍보) 2022 국방 AI 경진대회 (신청기간(일반인) : 9/30(금)~11/2(수) / 온라인 신청)
코드이력관리시스템  집중등록 기간(~10/21)
2022년 2학기 가디언 프로그램 안내
SW전공트랙 이수 신청 안내(~10/31)
★2022학년도 2학기 SW융합전공 장학생 모집 안내(신청일: 9/26~10/7)
2022 공개SW Week 특강 프로그램 신청 안내(~10/13)
LG CNS 하반기 신입사원 채용(9.5~9.26)
(홍보)SW중심대학 V-log 영상 공모전 '소중人의 탐구생활' (접수기간 : 9/5(월)~10/7(금)까지)
2022년 산학협력공동프로젝트 연구보조원 추가 모집
2022년 오픈소스SW 운영보조원 추가 모집 공고 (~9/15)
2022년 현장실습(인턴십) 체험수기 

In [10]:
from bs4 import BeautifulSoup
import requests

for i in range(1, 47):
    html = requests.get(f"https://sw7up.cbnu.ac.kr/community/notice?page={i}&limit=10&sort=-createdAt").text
    bsObject = BeautifulSoup(html, "html.parser")

    for link in bsObject.find_all('small')[2::3]:
        print(str(link).split('>')[1].split('.<')[0])
        res['date'].append(str(link).split('>')[1].split('.<')[0])

22.11.11
22.11.11
22.10.26
22.10.24
22.08.31
22.11.11
22.11.10
22.11.08
22.11.07
22.10.27
22.10.25
22.10.14
22.10.07
22.10.06
22.10.06
22.10.05
22.10.04
22.09.30
22.09.29
22.09.28
22.09.23
22.09.22
22.09.22
22.09.21
22.09.15
22.09.13
22.09.07
22.09.07
22.09.02
22.08.31
22.08.30
22.08.30
22.08.26
22.08.23
22.08.23
22.08.19
22.08.18
22.08.11
22.08.11
22.08.11
22.08.10
22.08.09
22.08.09
22.08.05
22.08.03
22.08.03
22.08.01
22.07.29
22.07.28
22.07.27
22.07.27
22.07.27
22.07.26
22.07.26
22.07.26
22.07.25
22.07.18
22.07.12
22.07.12
22.07.11
22.07.08
22.07.07
22.07.06
22.07.04
22.07.04
22.07.01
22.06.30
22.06.30
22.06.29
22.06.28
22.06.27
22.06.22
22.06.21
22.06.20
22.06.16
22.06.15
22.06.09
22.06.07
22.06.07
22.06.02
22.06.02
22.06.02
22.05.31
22.05.30
22.05.30
22.05.27
22.05.26
22.05.26
22.05.26
22.05.25
22.05.25
22.05.25
22.05.24
22.05.24
22.05.18
22.05.13
22.05.13
22.05.13
22.05.13
22.05.12
22.05.12
22.05.12
22.05.11
22.05.10
22.05.04
22.05.02
22.05.02
22.05.02
22.04.29
22.04.26
22.04.26
2

In [1]:
%pip install selenium
%pip install webdriver_manager
%pip install firebase_admin

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
browser = webdriver.Chrome(ChromeDriverManager().install())

for i in range(1,28):                                                                               #29페이지에 오류가 있어 url을 두개의 범위로 나누어 크롤링했다.
    browser.get(f"https://sw7up.cbnu.ac.kr/community/notice?page={i}&limit=10&sort=-createdAt")
    time.sleep(5)

    for i in range(10):
        elem = browser.find_elements(By.CLASS_NAME, 'card')
        elem[i].click()
        time.sleep(5)
        
        res['url'].append(browser.current_url)
        browser.back()
        time.sleep(5)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_8228\916816424.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
browser = webdriver.Chrome(ChromeDriverManager().install())

for i in range(30,47):                                                                              #29페이지에 오류가 있어 url을 두개의 범위로 나누어 크롤링했다.
    browser.get(f"https://sw7up.cbnu.ac.kr/community/notice?page={i}&limit=10&sort=-createdAt")
    time.sleep(5)

    for i in range(10):
        elem = browser.find_elements(By.CLASS_NAME, 'card')
        elem[i].click()
        time.sleep(5)
        
        res['url'].append(browser.current_url)
        browser.back()
        time.sleep(5)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_8228\916816424.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [8]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("<your private key>")
firebase_admin.initialize_app(cred)


In [12]:
print(res['title'][0])
print(res['date'][0])
print(res['url'][0])


2022년 SW중심대학사업단 Git-Github 기초/심화 특강 신청 안내(~11/24)
22.11.11
https://sw7up.cbnu.ac.kr/community/notice/636df9e59c91db44543ab74f


In [31]:
import pandas as pd

data = pd.DataFrame(res)
print(data)
data.to_csv('C:\\Users\\SAMSUNG\\Desktop\\crawl\\data.csv')         #크롤링 데이터를 csv 파일에 넣는다

                                                 title      date  \
0    2022년 SW중심대학사업단 Git-Github 기초/심화 특강 신청 안내(~11/24)  22.11.11   
1                      2022학년도 2학기 마일리지 장학금 제출 안내(필독★)  22.11.11   
2                                   코딩이력관리시스템 매뉴얼 다운로드  22.10.26   
3       2022년 2차 Solve-UpⓇ 프로그래밍 경진대회 참가 신청 안내(~11/23)  22.10.24   
4        2022학년도 2학기 마일리지 장학생 모집 안내 (신청일: 11/14~11/29)  22.08.31   
..                                                 ...       ...   
455             2020학년도 예비대학생을 위한 교양 3학점 인정 SW기초교육 안내문  20.01.16   
456                                    2020년 SW융합전공 안내  20.01.03   
457                                EASY 코스 교과목 수강신청 안내  20.01.03   
458                            "구글 엔지니어와의 만남" 일정 연기 안내  19.12.27   
459                                       구글 엔지니어와의 만남  19.12.20   

                                                   url  
0    https://sw7up.cbnu.ac.kr/community/notice/636d...  
1    https://sw7up.cbnu.ac.kr/community/notice/636d...  
2    https:/

In [11]:
import pandas as pd
res = pd.read_csv('./data.csv')
print(res)

     Unnamed: 0                                              title      date  \
0             0  2022년 SW중심대학사업단 Git-Github 기초/심화 특강 신청 안내(~11/24)  22.11.11   
1             1                    2022학년도 2학기 마일리지 장학금 제출 안내(필독★)  22.11.11   
2             2                                 코딩이력관리시스템 매뉴얼 다운로드  22.10.26   
3             3     2022년 2차 Solve-UpⓇ 프로그래밍 경진대회 참가 신청 안내(~11/23)  22.10.24   
4             4      2022학년도 2학기 마일리지 장학생 모집 안내 (신청일: 11/14~11/29)  22.08.31   
..          ...                                                ...       ...   
455         455             2020학년도 예비대학생을 위한 교양 3학점 인정 SW기초교육 안내문  20.01.16   
456         456                                    2020년 SW융합전공 안내  20.01.03   
457         457                                EASY 코스 교과목 수강신청 안내  20.01.03   
458         458                            "구글 엔지니어와의 만남" 일정 연기 안내  19.12.27   
459         459                                       구글 엔지니어와의 만남  19.12.20   

                                       

In [13]:
from firebase_admin import firestore

db = firestore.client()
for i in range(460):
    doc_ref = db.collection(u'sw7up').document(u'%d' % i)
    doc_ref.set({'title': res['title'][i], 
                'date': res['date'][i],
                'url': res['url'][i]})